In [4]:
# CV2 doesn't install well on the rasp pi so we will try and limit the usage of that library across the analysis so as not to introduce techniques that cannot be repeated on the feeder.   
# To the end, this notebook shares a code base with the feeder using the gcs.py and image_proc.py python files from the feeder project itself.  
try:
    import cv2
except:
    !pip install opencv-contrib-python
    import cv2

In [5]:
import pandas as pd
import gcs_inventory as gcsi
import matplotlib.pyplot as plt
import gcs
from io import BytesIO
from PIL import Image
import numpy as np
import image_proc

In [6]:
# set up gcs storage for future reference throughout
archive_web_storage = gcs.Storage(bucket_name='archive_jpg_from_birdclassifier')
nabirds_web_storage = gcs.Storage(bucket_name='nabirds_filtered')

# read the list of nabird files created with gcs_inventory locally and uploaded 
# saves some $$ avoiding a full list from gcs repeatedly
df_file_name = 'nabirds-jpg-list.csv'
try: 
    df_raw = pd.read_csv(df_file_name)  # reload existing file if it exists, delete it if you want to recreate it
    print(f'Loading existing file {df_file_name}')
except:
    df_raw = gcsi.get_nabirds_jpg_images(df_file_name)  # .01 per 1000 operations so about $0.80 per run, create from scratch
    print(f'Reading images from GCS....')

df_raw.to_csv(df_file_name, index=False)
print(f'Saved jpg list as {df_file_name}....')  # this save my be redundant if the file was loaded from disk vs. recreated from the gcs listing


AttributeError: module 'gcs_inventory' has no attribute 'get_nabirds_jpg_images'

In [ ]:
### helper functions  ###

# open cv does not install well on rasp pi, adding focal quality function here instead of image_proc
# this technique would need to be manually created in the image_proc lib for the pi
def focal_quality(img: Image.Image) -> float:
    laplacain_operator = cv2.Laplacian(np.array(image_proc.grayscale(img)), cv2.CV_64F)
    return laplacain_operator.var()